In [ ]:
from fastai import *
from fastai.vision import *

## <font color=#2196f3>从 Google 下载数据，训练</font>

### <font color=#2196f3>构造路径</font>

In [ ]:
folder = 'black'
file = 'urls_black.txt'

In [ ]:
folder = 'teddys'
file = 'urls_teddys.txt'

In [ ]:
folder = 'grizzly'
file = 'urls_grizzly.txt'

In [ ]:
path = Path('data/bears')
dest = path/folder
dest.mkdir(parents=True, exist_ok=True)

### <font color=#2196f3>下载图片</font>

In [ ]:
classes = ['teddys', 'grizzly', 'black']

In [ ]:
download_images(path/file, dest, max_pics=200)
#download_images(path/file, dest, max_pics=20, max_workers=0)

In [ ]:
for c in classes:
    print(c)
    verify_images(path/c, delete=True, max_work=8)


### <font color=#2196f3>查看数据</font>

In [ ]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train='.', valid_pct=0.2, ds_tfms=get_transforms(), size=224, num_workers=4).normalize(imagenet_stats)
data.classes

In [ ]:
data.show_batch(rows=3, figsize=(7, 8))

In [ ]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

### <font color=#2196f3>Train model</font>

In [ ]:
learn = create_cnn(data, models.resnet34, metrics=error_rate)
learn.fit_one_cycle(4)
learn.save('stage-1')

In [ ]:
learn.unfreeze()
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(2, max_lr=slice(3e-5, 3e-4))
learn.save('stage-2')

### <font color=#2196f3>Interpretaion</font>

In [ ]:
learn.load('stage-2')
interp = ClassificationInterpretation.from_larnner(learn)
interp.plot_confusion_matrix()

### <font color=#2196f3>数据清洗</font>

In [ ]:
from fastai.widgets import *

lesses, idxs = interp.top_losses()
top_loss_paths = data.valid_da.x[idxs]

In [ ]:
fd = FileDeleter(file_paths=top_loss_paths)

### <font color=#2196f3>Putting your model in production</font>

In [ ]:
data.classes

In [ ]:
# fastai.defaults.device = torch.device('cpu')
img = open_image(path/'black'/'0000021.jpg')
img

In [ ]:
# 此 cell 只做一次
classes = ['black', 'grizzly', 'teddys']
data2 = ImageDataBunch.single_from_classes(path, classes, tfms=get_transforms(), size=224).normalize(imagenet_state)
learn = create_cnn(data2, models.resnet34)
learn.load('stage-s')

In [ ]:
pred_class.pred_idx.outputs = learn.predict(img)
pred_class

## <font color=#2196f3>2.6 效果不好的一些情况</font>

### <font color=#2196f3>Learn rate(LR) too high  学习率太高</font>

In [ ]:
learn = create_cnn(data, models.resnet34, metrics=error_rate)

# 默认学习率是0.003，大多时候都比较好用，这里用0.5，结果是验证损失高到离谱，通常是低于1的，如果验证损失过高，仅靠轮次去弥补是无济于事的
learn.fit_one_cycle(1, max_lr=0.5)

### <font color=#2196f3>Learn rae(LR)too low 学习率过低</font>

In [ ]:
learn = create_cnn(data, models.resnet34, metrics=error_rate)

# 学习率设为0.00001，误差表现得越来越好，但是变化的非常非常慢，在输出中你还会发现，训练损失会高于验证损失，意味着还没有充分拟合，要么是学习率太低要么是轮次太少，
learn.fit_one_cycle(5, max_lr=1e-5)

# 如果调用learn.recorder()函数 可以把这个过程绘成图片，如果看到这个下降的非常慢很可能是学习率太低了
learn.recorder.plot_losses()

### <font color=#2196f3>too few epochs 轮次太少</font>

In [ ]:
learn = create_cnn(data, models.resnet34, metrics=error_rate)

# 从输出结果中对比一下训练损失和验证损失，训练损失要比验证损失高得多，过少的轮次和过低的学习率结果非常相似
learn.fit_one_cycle(1)

### <font color=#2196f3>too many epochs 轮次太多</font>

In [ ]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train='.', valid_pct=0.9, bs=32, ds_tfms=get_transforms(do_flip=False, max_rotate=0, max_zoom=1, max_lighting=0, max_warp=0), size=224, num_workers=4).normalize(imagenet_stats) 

learn = create.cnn(data, models.resnet50, metrics=error_rate, ps=0, wd=0)
learn.unfreeze()

learn.fit_one_cycle(40, slice(1e-6, 1e-4))     # 轮次太多，会导致过拟合

## <font color=#2196f3>2.9 创建一个简单的线性模型</font>

In [ ]:
%matplotlib inline
from fastai import *

n = 100

x = torch.ones(n, 2)      # nx2的二阶张量，行为n列为2
x[:, 0].uniform_(-1, 1)
x[:5]

a = tensor(3., 2.)
y = x@a + torch.rand(n)

plt.scatter(x[:, 0], y)

### $ \frac{\sum(\hat{y}-y)^2}{n} $

In [ ]:
# 损失函数（均方差）
def mse(y_hat, y):
    return ((y_hat-y)**2).mean()

In [ ]:
a = tensor(-1., 1)    # 猜测参数初始值
y_hat = x@a
mse(y_hat, y)

In [ ]:
plt.scatter(x[:, 0], y)
plt.scatter(x[:, 0], y_hat)

In [ ]:
a = nn.Parameter(a)
a

In [ ]:
def update():
    y_hat = x@a
    loss = mse(y, y_hat)
    if t % 10 == 0:
        print(loss)
    loss.backward()  # 计算梯度
    
    # 关闭梯度计算
    with torch.no_grad():
        a.sub_(lr * a.grad)  # lr(learning rate学习率，梯度存放于a.grad种)
        a.grad.zero_()

In [ ]:
lr = 1e-1
for t in range(100):
    update()

In [ ]:
plt.scatter(x[:, 0], y)
plt.scatter(x[:, 0], x@a)

## Animate it!

In [ ]:
from matplotlib import animation, rc
rc('animation', html='html5')

a = nn.Parameter(tensor(-1., 1))
fig = plt.figure()
plt.scatter(x[:, 0], y, c='orange')
line, = plt.plot(x[:, 0], x@a)
plt.close()

def animate(i):
    update()
    line.set_ydata(x@a)
    return line

animation.FuncAnimation(fig, animate, np.arange(0, 100), interval=20)

Learning rate 学习率 乘以 梯度值用来更新权重
Epoch 轮次
Minibatch 随机数据点
SGD 就是使用mini-batches方法的梯度下降法
正则化会避免过拟合